## Desafío entregable #6: Descarga de datos desde APIs públicas

En el presente desafío se obtendrá información sobre canciones lanzadas entre los años 1980 y 2022 procedentes de la plataforma de streamming de música Spotify a traves de su Web API. Por medio del paquete Spotipy podremos comunicarnos con la API empleando Python. 

https://developer.spotify.com/documentation/web-api/

https://spotipy.readthedocs.io/en/2.22.1/

### Instalamos e importamos los paquetes

Instalamos el paquete Spotipy y lo importamos junto con otros paquetes que serán de utilidad

In [1]:
# pip install spotipy --upgrade

In [2]:
import json
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

Para poder acceder al Spotify Web API se requiere de registar una cuenta en Spotify for Developers. Esta última es gratuita, no requiere de pagar el servicio Spotify Premium. Este paso es importante porque permite obtener las API Keys: ClientID y Client Secret, con las cuales acceder a la API. Estas claves son secretas y no serán compartidas.

https://developer.spotify.com/

In [3]:
cid = 'xxxxxxxxx'
secret = 'xxxxxxxxx'

Con las API Keys procedemos a acceder Spotify mediante la libreria Spotipy

In [4]:
client_credentials_manager = SpotifyClientCredentials(client_id = cid, client_secret = secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

Se usarán dos comandos para extraer información de la API: search y audio_features, los cuales ameritan una breve descripción.

- search permite buscar canciones que cumplan un determinado criterio, el cual puede ser el nombre del artista, el género o el año de lanzamiento de la canción. Este comando está limitado a entregar información de un máximo de 50 canciones por vez (limit) de una lista de hasta 1000 canciones (offset). Se pueden obtener más datos haciendo uso de un loop for.

https://developer.spotify.com/documentation/web-api/reference/#/operations/search

- audio_features entrega datos de una determinada canción sobre una serie de parámetros numéricos que dan información sobre el volumen, la energía que transmite, el carácter instrumental, la presencia de palabra hablada o cantanda, entre otras. Este comando requiere del Spotify ID de la canción de interés.

https://developer.spotify.com/documentation/web-api/reference/#/operations/get-audio-features

### Adquisición de datos

En primer lugar se obtiene por medio del comando search una lista de canciones entre los años 1980 y 2022. Se obtiene el nombre del artista, el nombre de la cancion y la popularidad de la misma. Además cada cancion posee un ID y un URI.

In [28]:
year, artist_name, track_name, popularity, track_id, track_uri = [], [], [], [], [], []

for y in range(1980, 2023):
    for i in range(0, 200, 50):
        track_results = sp.search(q = f'year:{y}', type = 'track', limit = 50, offset = i)
        
        for i, t in enumerate(track_results['tracks']['items']):
            year.append(int(y))
            artist_name.append(t['artists'][0]['name'])
            track_name.append(t['name'])
            track_id.append(t['id'])
            track_uri.append(t['uri'])
            popularity.append(t['popularity'])

Con la mencionada lista se arma el dataset provisorio df_tracks. Este dataset será luego utilizado para armar el dataset definitivo.

In [29]:
df_tracks = pd.DataFrame({'year':year, 'artist':artist_name, 'track':track_name, 'id':track_id, 'popularity':popularity})

print(df_tracks.shape[0])
df_tracks.head()

8600


,year,artist,track,id,popularity
0,1980,AC/DC,You Shook Me All Night Long,2SiXAy7TuUkycRVbbWDEpo,80
1,1980,AC/DC,Back In Black,08mG3Y1vljYA6bvDt4Wqkj,82
2,1980,Daryl Hall & John Oates,You Make My Dreams (Come True),4o6BgsqLIBViaGVbx5rbRk,77
3,1980,Queen,Another One Bites The Dust - Remastered 2011,57JVGBtBLCfHw2muk5416J,72
4,1980,Bob Marley & The Wailers,Could You Be Loved,5O4erNlJ74PIF6kGol1ZrC,79


Empleando los URI de cada canción se procedió a obtener los parámetros numéricos de cada canción por medio del comando audio_features. Dichos parámetros son almacenados en un diccionario.

In [31]:
audio_features = {}
    
for idx, i in enumerate(track_uri):
    audio_features[f'{idx}'] = sp.audio_features(i)[0]
    print('Progreso = {} %'.format(round(idx*100/(df_tracks.shape[0]), 1)), end = '\r')

El comando audio_feature solicita (request) información a la API sobre una determinada canción, haciendo cada vez una conexión con Spotify. Esto lo repite para cada una de las 8600 canciones, estoy hace el proceso un tanto lento y susceptible a fluctuacciones en el ancho de banda y suele incurrir en errores por pérdida de conexion. 

Con el diccionario audio_features se arma un segundo dataset provisorio, df_audio_features.

In [32]:
df_audio_features = pd.DataFrame.from_dict(audio_features).T 

drop = ['type', 'uri', 'track_href', 'analysis_url']
df_audio_features.drop(drop, axis = 1, inplace = True)

print(df_audio_features.shape)
df_audio_features.head()

(8600, 14)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature
0,0.532,0.767,7,-5.509,1,0.0574,0.00287,0.000513,0.39,0.755,127.361,2SiXAy7TuUkycRVbbWDEpo,210173,4
1,0.31,0.7,9,-5.678,1,0.047,0.011,0.00965,0.0828,0.763,188.386,08mG3Y1vljYA6bvDt4Wqkj,255493,4
2,0.751,0.501,5,-12.151,1,0.0551,0.234,0.112,0.0467,0.902,167.057,4o6BgsqLIBViaGVbx5rbRk,190627,4
3,0.933,0.528,5,-6.472,0,0.162,0.112,0.329,0.163,0.756,109.975,57JVGBtBLCfHw2muk5416J,214653,4
4,0.916,0.72,0,-8.548,1,0.1,0.36,0.00016,0.0958,0.76,103.312,5O4erNlJ74PIF6kGol1ZrC,237000,4


Finalmente, unimos los dos datasets provisorios por medio del comando merge de Pandas, el cual opera de manera similar al comando JOIN del lenguaje SQL. El dataset resultante es el definitivo y es el que contiene todos los datos de interés que fueron solicitados a la Web API de Spotify.

In [33]:
df = pd.merge(df_tracks, df_audio_features, on = 'id', how = 'inner') 
print('Dimensiones del dataset:', df_audio_features.shape) 
df.head()

Dimensiones del dataset: (8600, 14)


,year,artist,track,id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,1980,AC/DC,You Shook Me All Night Long,2SiXAy7TuUkycRVbbWDEpo,80,0.532,0.767,7,-5.509,1,0.0574,0.00287,0.000513,0.39,0.755,127.361,210173,4
1,1980,AC/DC,Back In Black,08mG3Y1vljYA6bvDt4Wqkj,82,0.31,0.7,9,-5.678,1,0.047,0.011,0.00965,0.0828,0.763,188.386,255493,4
2,1980,Daryl Hall & John Oates,You Make My Dreams (Come True),4o6BgsqLIBViaGVbx5rbRk,77,0.751,0.501,5,-12.151,1,0.0551,0.234,0.112,0.0467,0.902,167.057,190627,4
3,1980,Queen,Another One Bites The Dust - Remastered 2011,57JVGBtBLCfHw2muk5416J,72,0.933,0.528,5,-6.472,0,0.162,0.112,0.329,0.163,0.756,109.975,214653,4
4,1980,Bob Marley & The Wailers,Could You Be Loved,5O4erNlJ74PIF6kGol1ZrC,79,0.916,0.72,0,-8.548,1,0.1,0.36,0.00016,0.0958,0.76,103.312,237000,4


El dataset es guardado como un archivo .csv

In [34]:
df.to_csv('Spotify_API.csv')